In [2]:
# /* Q1 */
# SELECT * FROM Facilities WHERE membercost > 0.0

# /* Q2 */
# SELECT COUNT(facid) FROM Facilities where membercost = 0.0
# /* There are 4 facilities */

# /* Q3 */
# SELECT facid, name, membercost, monthlymaintenance FROM Facilities WHERE membercost > 0.0 AND membercost < 0.2 * monthlymaintenance

# /* Q4 */
# SELECT * FROM Facilities WHERE facid in(1,5)

# /* Q5 */
# SELECT name, monthlymaintenance, CASE WHEN monthlymaintenance > 100 then 'expensive' when monthlymaintenance < 100 then 'cheap' end as priceLabel from Facilities

# /* Q6 */
# SELECT firstname, surname, joindate
# FROM Members
# WHERE joindate = (SELECT MAX(joindate) FROM Members);

# /* Q7 */
# SELECT DISTINCT
#     /*CONCAT(m.firstname, ', ', m.surname, ', ', f.name) AS out_name */
#     m.firstname, m.surname, f.name
# FROM Bookings AS b
# INNER JOIN Facilities AS f ON b.facid = f.facid
# INNER JOIN Members AS m ON b.memid = m.memid
# WHERE f.name LIKE 'Tennis Court%'
# ORDER BY m.firstname, m.surname;

# /* Q8 */
# SELECT 
#     f.name AS facility_name,
#     /* CONCAT(m.firstname, ',' , m.surname) */
#     m.firstname, m.surname,
#     CASE 
#         WHEN b.memid = 0 THEN b.slots * f.guestcost
#         ELSE b.slots * f.membercost
#     END AS total_cost
# FROM 
#     Bookings AS b
#     INNER JOIN Facilities AS f ON b.facid = f.facid
#     INNER JOIN Members AS m ON b.memid = m.memid
# WHERE 
#     date(b.starttime) = '2012-09-14' 
#     AND (
#         (b.memid = 0 AND b.slots * f.guestcost > 30) 
#         OR (b.memid != 0 AND b.slots * f.membercost > 30)
#     )
# ORDER BY total_cost DESC;

# /* Q9 */
# SELECT 
#     f.name AS facility_name,
#     /*CONCAT(m.firstname, ', ', m.surname) AS member_name */
#     m.firstname, m.surname,
#     sub.total_cost
# FROM 
#     Members AS m
#     INNER JOIN (
#         SELECT 
#             b.memid,
#             b.facid,
#             CASE 
#                 WHEN b.memid = 0 THEN b.slots * f.guestcost
#                 ELSE b.slots * f.membercost
#             END AS total_cost
#         FROM 
#             Bookings AS b
#             INNER JOIN Facilities AS f ON b.facid = f.facid
#         WHERE 
#             DATE(b.starttime) = '2012-09-14' 
#             AND (
#                 (b.memid = 0 AND b.slots * f.guestcost > 30) 
#                 OR (b.memid != 0 AND b.slots * f.membercost > 30)
#             )
#     ) AS sub ON m.memid = sub.memid
#     INNER JOIN Facilities AS f ON sub.facid = f.facid
# ORDER BY sub.total_cost DESC;


import sqlite3
from sqlite3 import Error

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn, query):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    cur.execute(query)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)
    
    

def main():
    database = r"C:\Users\Josh Zuo\Downloads\sqlite_db_pythonsqlite.db"
    
    query1 = """
        SELECT 
    f.name AS facility_name,
    SUM(
        CASE 
            WHEN b.memid = 0 THEN b.slots * f.guestcost
            ELSE b.slots * f.membercost
        END
    ) AS total_revenue
FROM 
    Facilities AS f
    INNER JOIN Bookings AS b ON f.facid = b.facid
GROUP BY 
    f.name
HAVING 
    total_revenue < 1000
ORDER BY 
    total_revenue;
        """
    
    query2 = """
        SELECT 
    m.surname AS member_surname,
    m.firstname AS member_firstname,
    r.surname AS recommender_surname,
    r.firstname AS recommender_firstname
    FROM 
    Members AS m
    LEFT JOIN Members AS r ON m.recommendedby = r.memid
ORDER BY 
    m.surname, 
    m.firstname;
        """
    query3 = """
    SELECT 
    f.name AS facility_name,
    COUNT(b.bookid) AS member_usage
FROM 
    Facilities AS f
    INNER JOIN Bookings AS b ON f.facid = b.facid
WHERE 
    b.memid != 0
GROUP BY 
    b.memid
ORDER BY 
    member_usage DESC;
        """
    query4 = """
    SELECT 
    f.name AS facility_name, strftime('%m', b.starttime) as month_id,
    COUNT(b.bookid) AS member_usage
FROM 
    Facilities AS f
    INNER JOIN Bookings AS b ON f.facid = b.facid
WHERE 
    b.memid != 0
GROUP BY 
    month_id
ORDER BY 
    member_usage DESC;
        """
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("Q10:")
        select_all_tasks(conn, query1)
        print("Q11:")
        select_all_tasks(conn, query2)
        print("Q12:")
        select_all_tasks(conn, query3)
        print("Q13:")
        select_all_tasks(conn, query4)
 
 
if __name__ == '__main__':
    main()

2.6.0
Q10:
('Table Tennis', 180)
('Snooker Table', 240)
('Pool Table', 270)
Q11:
('Bader', 'Florence', 'Stibbons', 'Ponder')
('Baker', 'Anne', 'Stibbons', 'Ponder')
('Baker', 'Timothy', 'Farrell', 'Jemima')
('Boothe', 'Tim', 'Rownam', 'Tim')
('Butters', 'Gerald', 'Smith', 'Darren')
('Coplin', 'Joan', 'Baker', 'Timothy')
('Crumpet', 'Erica', 'Smith', 'Tracy')
('Dare', 'Nancy', 'Joplette', 'Janice')
('Farrell', 'David', None, None)
('Farrell', 'Jemima', None, None)
('GUEST', 'GUEST', None, None)
('Genting', 'Matthew', 'Butters', 'Gerald')
('Hunt', 'John', 'Purview', 'Millicent')
('Jones', 'David', 'Joplette', 'Janice')
('Jones', 'Douglas', 'Jones', 'David')
('Joplette', 'Janice', 'Smith', 'Darren')
('Mackenzie', 'Anna', 'Smith', 'Darren')
('Owen', 'Charles', 'Smith', 'Darren')
('Pinker', 'David', 'Farrell', 'Jemima')
('Purview', 'Millicent', 'Smith', 'Tracy')
('Rownam', 'Tim', None, None)
('Rumney', 'Henrietta', 'Genting', 'Matthew')
('Sarwin', 'Ramnaresh', 'Bader', 'Florence')
('Smith',